In [ ]:
#Installing necessary dependences:
#!pip install -q keras-nlp

In [ ]:
#!pip list

In [ ]:
#imports:
import tensorflow as tf
import tensorflow_text as tf_text
import keras
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, MultiHeadAttention, Softmax, TextVectorization, LayerNormalization
from keras import Model, Input
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [ ]:
import keras_nlp

In [ ]:
#Preprocessing Data
filepath = os.path.dirname(os.path.dirname(os.path.abspath("."))) + "/Datasets"
filepath = re.sub(r'\\', '/', filepath)
print(filepath)
X1 = pd.read_csv(filepath + "/Tatobea Project" + '/'+ "spa" +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
X1 = X1[X1.notnull().all(axis = 1)]

In [ ]:
print(X1)

In [ ]:
X1['Translated'] = "<ES> " + X1['Translated']

In [ ]:
print(X1)

In [ ]:
train_x = pd.DataFrame()

In [ ]:
for d in [x[0] for x in os.walk(filepath + "/DGT-TM")]:
    print(d)
    print(d[-6:])
    #Ignores TMX files
    if d[-1] == ")" or d[-6:] == "DGT-TM":
        continue
    #This code actually works, however due to the dataset it takes 1+ hour to run.
    #Because of this, at least for now I'll be restricting it to just the EN-ES folder.
    te = True
    if d[-5:] != "EN-ES":
        te = False
        continue
    
    fp = re.sub(r'\\', '/', d)
    E = fp.split("/")[-1].split("-")[0]
    print(E)
    T = fp.split("/")[-1].split("-")[1]
    print(T)
    print(d + '/'+ E + "-" + T +'.txt')
    
    temp = pd.read_csv(d + '/'+ E + "-" + T +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
    temp = temp[temp.notnull().all(axis = 1)]
    temp['Translated'] = "<" + T + "> " + temp['Translated']
    
    print(temp.head())
    if te:
        X1 = pd.concat([X1, temp])
    else:
        train_x = pd.concat([train_x, temp])


In [ ]:
Y1 = pd.DataFrame()
Y1['Translated'] = X1.pop("Translated")


In [ ]:
print(Y1)

In [ ]:
train_x1, test_x, train_y1, test_y = train_test_split(X1, Y1, test_size=0.5, random_state=42)

In [ ]:
try:
    train_y = train_x.pop("Translated")
except:
    #train_x doesn't have any portions, so we'll just describe it as a normal Dataframe
    train_y = pd.DataFrame()

In [ ]:
train_x = pd.concat([train_x, train_x1])

In [ ]:
train_y = pd.concat([train_y, train_y1])

In [ ]:
del train_x1
del train_y1
del X1
del Y1

In [ ]:
print(train_x)
print(train_y)
print(test_x)
print(test_y)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
#train_x = train_x.astype(str)
#test_x = test_x.astype(str)

In [ ]:
#X = pd.concat([train_x, test_x])
#Y = pd.concat([train_y, test_y])
#Y_SP = Y[Y['Translated'].str.contains("<ES>")]

In [ ]:
#max_padding_series_org = X['English'].str.split(" ")
#max_padding_len_org = max_padding_series_org.str.len()
#print(max_padding_len_org)
#max_padding_size_org = max_padding_len_org.max() + 2

In [ ]:
#The +2 is necessary because of the [START] and [END] tokens
#print(max_padding_size_org)

In [ ]:
#max_padding_series_tar = Y_SP['Translated'].str.split(" ")
#max_padding_len_tar = max_padding_series_tar.str.len()
#print(max_padding_len_tar)
#max_padding_size_tar = max_padding_len_tar.max() + 2

In [ ]:
#print(max_padding_size_tar)

In [ ]:
#Code used to find the length of 
#vectorizer_org = CountVectorizer()
#count_org = vectorizer_org.fit_transform(X['English'])
#vectorizer_tar = CountVectorizer()
#count_tar = vectorizer_tar.fit_transform(Y_SP['Translated'])
#print(count_org, count_tar)

In [ ]:
#print(count_org.shape)

In [ ]:
#print(len(vectorizer_org.get_feature_names_out()))

In [ ]:
#print(len(vectorizer_tar.get_feature_names_out()))

In [ ]:
#The total number of words in the English corpus seems to be 285636.
#The total number of words in the Spanish corpus seems to be 340903.
org_length = 285636
tar_length = 340903

In [ ]:
#Need to tokenize train_x, test_x, train_y, test_y
#This code creates the vocabulary and saves it to a text file.
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_org_args = dict(
    # The target vocabulary size
    vocab_size = org_length,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

bert_vocab_tar_args = dict(
    # The target vocabulary size
    vocab_size = tar_length,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [ ]:
print(bert_tokenizer_params)


In [ ]:
#X_tensor = tf.convert_to_tensor(X['English'])

In [ ]:
#print(X_tensor.shape)

In [ ]:
#X_dataset = tf.data.Dataset.from_tensor_slices(X_tensor)

In [ ]:
#en_vocab = bert_vocab.bert_vocab_from_dataset(
#    X_dataset.batch(1000).prefetch(2),
#    **bert_vocab_org_args
#)

In [ ]:
#with open("engvocab.txt", 'w', encoding="utf-8") as f:
#    for token in en_vocab:
#      print(token, file=f)

In [ ]:
Y_SP_tensor = tf.convert_to_tensor(Y_SP['Translated'])

In [ ]:
#Y_SP_dataset = tf.data.Dataset.from_tensor_slices(Y_SP_tensor)

In [ ]:
#es_vocab = bert_vocab.bert_vocab_from_dataset(
#    Y_SP_dataset.batch(1000).prefetch(2),
#    **bert_vocab_tar_args
#)

In [ ]:
#with open("esvocab.txt", 'w', encoding="utf-8") as f:
#    for token in es_vocab:
#      print(token, file=f)

In [ ]:
es_tokenizer = tf_text.BertTokenizer('esvocab.txt', **bert_tokenizer_params)
en_tokenizer = tf_text.BertTokenizer('engvocab.txt', **bert_tokenizer_params)

In [ ]:
x_train_tensor = en_tokenizer.tokenize(train_x['English'])
x_train_tensor = x_train_tensor.merge_dims(-2,-1)
x_train_tensor = x_train_tensor.to_tensor(default_value=0)



In [ ]:
x_test_tensor = en_tokenizer.tokenize(test_x['English'])
x_test_tensor = x_test_tensor.merge_dims(-2,-1)
x_test_tensor = x_test_tensor.to_tensor(default_value=0)

In [ ]:
y_train_tensor = es_tokenizer.tokenize(train_y['Translated'])
y_train_tensor = y_train_tensor.merge_dims(-2,-1)




In [ ]:
#0 is the ID for the [PAD] token
y_train_tensor = y_train_tensor.to_tensor(default_value=0)

In [ ]:
y_test_tensor = es_tokenizer.tokenize(test_y['Translated'])
y_test_tensor = y_test_tensor.merge_dims(-2,-1)


In [ ]:
y_test_tensor = y_test_tensor.to_tensor(default_value=0)

In [ ]:
print(x_train_tensor.shape)
print(x_test_tensor.shape)
print(y_train_tensor.shape)


In [ ]:
print(y_test_tensor.shape)

In [ ]:
import gc
gc.collect()

In [ ]:
#(2917819, 611)
#(2917820, 777)
#(2917819, 622)
#(2917820, 794)

In [ ]:
org_length = 777
tar_length = 794

In [ ]:
STOP

In [ ]:
#Actual Model:
#For now, let's start by creating an exact copy of the attention is all you need model.
#We can always make changes later if we want to.
i = Input(shape=(1,))

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#TLayer = TextVectorization(output_sequence_length = max_padding_size, vocabulary = m, output_mode = "int")
#inp = TextVectorization(output_sequence_length = max_padding_size)

In [ ]:
#inp = TLayer(i)

In [ ]:
#print(inp)

In [ ]:
#inp.adapt(train_x)

In [ ]:
#dir(inp)

In [ ]:
#print(inp.get_vocabulary())

In [ ]:
#f = open("textvectorvocap.txt", "w", encoding="utf-8")
#f.write("*".join(inp.get_vocabulary()))
#f.close()

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#print(m == inp.get_vocabulary())

In [ ]:
#print(inp.vocabulary_size())

In [ ]:
#train_x_vec = inp.predict(train_x)
#test_x_vec = inp.predict(test_x)

In [ ]:
#print(i.__call__())

In [ ]:
em = Embedding(input_dim = TLayer.vocabulary_size(), output_dim = 512)(inp)

In [ ]:
posen = keras_nlp.layers.SinePositionEncoding()(em)

In [ ]:
mh1 = MultiHeadAttention(num_heads=3, key_dim=512, value_dim=512)

In [ ]:
mho1 = mh1(posen, posen, posen)

In [ ]:
no1 = LayerNormalization()
ou1 = no1(mho1+posen)

In [ ]:
fe1 = Dense(512)(ou1)
ou2 = no1(ou1+fe1)

In [ ]:

lin = Dense(512)(ou2)

In [ ]:
s = Softmax()(lin)

In [ ]:
model = Model(inputs=i, outputs=s)
print(model.summary())

In [ ]:
#Need to figure out metrics for language translation